# Úkol č. 3 - předzpracování dat a binární klasifikace

  * V rámci tohoto úkolu se musíte vypořádat s příznaky, které jsou různých typů.
  * Před tím, než na nich postavíte predikční model, je třeba je nějakým způsobem převést do číselné reprezentace.
    
> **Úkoly jsou zadány tak, aby Vám daly prostor pro invenci. Vymyslet _jak přesně_ budete úkol řešit, je důležitou součástí zadání a originalita či nápaditost bude také hodnocena!**

## Zdroj dat

Budeme se zabývat predikcí přežití pasažérů Titaniku.
K dispozici máte trénovací data v souboru **data.csv** a data na vyhodnocení v souboru **evaluation.csv**.

#### Seznam příznaků:
* survived - zda přežil, 0 = Ne, 1 = Ano, **vysvětlovaná proměnná**, kterou chcete predikovat
* pclass - Třída lodního lístku, 1 = první, 2 = druhá, 3 = třetí
* name - jméno
* sex - pohlaví
* age - věk v letech
* sibsp	- počet sourozenců / manželů, manželek na palubě
* parch - počet rodičů / dětí na palubě
* ticket - číslo lodního lístku
* fare - cena lodního lístku
* cabin	- číslo kajuty
* embarked	- místo nalodění, C = Cherbourg, Q = Queenstown, S = Southampton
* home.dest - Bydliště/Cíl

## Pokyny k vypracování

**Základní body zadání**, za jejichž (poctivé) vypracování získáte **10 bodů**:
  * V Jupyter notebooku načtěte data ze souboru **data.csv**. Vhodným způsobem si je rozdělte na trénovací, testovací a případně i validační množinu.
  * Projděte si jednotlivé příznaky a transformujte je do vhodné podoby pro použití ve vybraném klasifikačním modelu.
  * Podle potřeby si můžete vytvářet nové příznaky (na základě existujících), například tedy můžete vytvořit příznak měřící délku jména. Některé příznaky můžete také úplně zahodit.
  * Nějakým způsobem se vypořádejte s chybějícími hodnotami.
  * Následně si vyberte vhodný klasifikační model z přednášek. Najděte vhodné hyperparametry a určete jeho přesnost (accuracy) na trénovací množině. Také určete jeho přesnost na testovací množině.
  * Načtěte vyhodnocovací data ze souboru **evaluation.csv**. Napočítejte predikce pro tyto data (vysvětlovaná proměnná v nich již není). Vytvořte **results.csv** soubor, ve kterém tyto predikce uložíte do dvou sloupců: ID, predikce přežití. Tento soubor nahrajte do repozitáře.

**Další body zadání** za případné další body  (můžete si vybrat, maximum bodů za úkol je každopádně 15 bodů):
  * (až +5 bodů) Aplikujte všechny klasifikační modely z přednášek a určete (na základě přesnosti na validační množině), který je nejlepší. Přesnost tohoto nejlepšího modelu odhadněte pomocí testovací množiny. K predikcím na vyhodnocovacích datech využijte tento model.
  * (až +5 bodů) Zkuste použít nějaké (alespoň dvě) netriviální metody doplňování chybějících hodnot u věku. Zaměřte na vliv těchto metod na přesnost predikce výsledného modelu. K predikcím na vyhodnocovacích datech využijte ten přístup, který Vám vyjde jako nejlepší.

## Poznámky k odevzdání

  * Řiďte se pokyny ze stránky https://courses.fit.cvut.cz/MI-PDM/homeworks/index.html.
  * Odevzdejte nejen Jupyter Notebook, ale i _csv_ soubor(y) s predikcemi pro vyhodnocovací data.
  * Opravující Vám může umožnit úkol dodělat či opravit a získat tak další body. První verze je ale důležitá a bude-li odbytá, budete za to penalizováni**

In [ ]:
import pandas as pd

import sklearn

from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import StandardScaler

In [ ]:
def process_data(file, y_col=""):
    data = pd.read_csv(file)

    # Drop columns with many missing data
    data = data.drop(["cabin", "home.dest"], axis=1)

    # Convert sex to indexes
    sex_idx = data["sex"].astype("category").cat.codes
    data.insert(5, "sex_idx", sex_idx, True)

    # Convert embarked to indexes
    embarked_idx = data["embarked"].astype("category").cat.codes
    data.insert(11, "embarked_idx", embarked_idx, True)

    # Remove unnecessary column
    data = data.drop(["ID", "name", "sex", "ticket", "fare", "embarked"], axis=1)

    # Fix missing data
    data = data.fillna(data.mean())
    
    X = data
    Y = []
    
    if y_col != "":
        Y = data[y_col]
        X = data.drop(y_col, axis=1)
    
    # Standardization
    X = StandardScaler().fit_transform(X)
    
    return (X, Y)

In [ ]:
X, Y = process_data("data.csv", "survived")

# Split data to train and validation
from sklearn.model_selection import train_test_split
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.2, random_state=42)
Xtrain, Xval, Ytrain, Yval = train_test_split(Xtrain, Ytrain, test_size=0.2, random_state=42)

In [ ]:
# Prepare callback for showing the progress
from tensorflow.keras.callbacks import Callback
class NEpochsLogger(Callback):
    def __init__(self, display):
        self.display = display

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.display == 0:
            print("Epoch {}/{}".format(epoch, self.params["epochs"]))
            print(logs)

In [ ]:
model = Sequential()
model.add(Dense(120, activation='relu', kernel_initializer='random_normal', input_dim=Xtrain.shape[1]))
model.add(keras.layers.Dropout(0.2))
model.add(Dense(60, activation='relu', kernel_initializer='random_normal'))
model.add(Dense(1, activation='sigmoid', kernel_initializer="random_normal"))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
#model.summary()

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

def train_network(patience):
    each = int(patience / 3)
    return model.fit(Xtrain, Ytrain,
          validation_data=(Xval, Yval),
          batch_size=10,
          epochs=100,
          verbose=False,
          callbacks=[EarlyStopping(monitor='val_loss', patience=patience), NEpochsLogger(each)]
         )

train_network(10)
train_network(3)

In [ ]:
test_loss, test_acc = model.evaluate(Xtest, Ytest)
print('Test loss:', test_loss)
print('Test accuracy:', test_acc)

In [ ]:
Xeval, _ = process_data("evaluation.csv")
Yeval = model.predict_classes(Xeval)
Yeval.sum()

In [ ]:
evaluation_data = pd.read_csv("evaluation.csv")
ids = evaluation_data["ID"]
survived = pd.Series(Yeval.flatten())
solution = pd.DataFrame({'ID': ids, 'Survived': survived})
# This overrides the file in the repo
# solution.to_csv("results.csv")

In [ ]:
# Pouze pro porovnání
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
logreg.fit(Xtrain, Ytrain)
Xeval, _ = process_data("evaluation.csv")
print("Acc:", logreg.score(Xtest, Ytest))
logreg.predict(Xeval).sum()